<a href="https://colab.research.google.com/github/robertoricci/Analise-Financeira/blob/main/NOTEBOOK_ETL_B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalar bibliotecas necessarias 
!pip install SQLAlchemy
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from base64 import b64encode
import pandas as pd
import sqlalchemy as sql

In [ ]:
#criar conexão com o banco Postgres

conn = 'suaconexao'
sql_conn = sql.create_engine(conn)

In [ ]:
# Função para obter empresas do IBOV ou IFIX
def getEmpIndex(seg=2,index='IBOV'):
    # cria os parâmetros 
    # index: IBOV 
    #        IFIX

    #segment:1 informações sem segmento
    #       :2 informações com segmento

    params = {"language":"pt-br","pageNumber":1,"pageSize":200,"index":index,"segment":seg}
    
    # codifica os parâmetros em base64
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params)
    string = string.decode()
   #{"language":"pt-br","pageNumber":1,"pageSize":120,"index":"IBOV","segment":"1"}

    r = requests.get('https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/'+ string)
    return r.json()

In [ ]:
index = getEmpIndex()

In [ ]:
df_ibov = pd.DataFrame.from_dict(index['results'])

In [ ]:

#df_ibov[['segment','cod','asset','type']] = df_ibov[['segment','cod','asset','type']].astype(str)
#df_ibov['part'].apply(lambda x: float(x.replace(',', '.')))
#df_ibov['partAcum'].apply(lambda x: float(x.replace(',', '.')))
#df_ibov['theoricalQty'].apply(lambda x: float(x))
# df_ibov['segment'] = df_ibov['segment'].astype(str)
# df_ibov['cod'] = df_ibov['cod'].astype(str)
# df_ibov['asset'] =df_ibov['asset'].astype(str)
# df_ibov['type'] = df_ibov['type'].astype(str)
df_ibov['part']= df_ibov['part'].str.replace(',', '.',regex=True).astype(float)
df_ibov['partAcum'] = df_ibov['partAcum'].str.replace(',', '.',regex=True).astype(float)
df_ibov['theoricalQty'] = df_ibov['theoricalQty'].str.replace('.', '',regex=True).astype(int)

In [ ]:
df_ibov.dtypes

segment          object
cod              object
asset            object
type             object
part            float64
partAcum        float64
theoricalQty      int64
dtype: object

In [ ]:
#altera tipos das colunas

In [ ]:
df_ibov.set_index('cod')

,segment,asset,type,part,partAcum,theoricalQty
cod,,,,,,
WEGE3,Bens Indls / Máqs e Equips,WEG,ON NM,2.811,2.811,1481593024
EMBR3,Bens Indls / Mat Transporte,EMBRAER,ON NM,0.478,0.478,734632205
AZUL4,Bens Indls/Transporte,AZUL,PN N2,0.198,2.106,327646296
CCRO3,Bens Indls/Transporte,CCR SA,ON ED NM,0.607,2.106,1115681435
ECOR3,Bens Indls/Transporte,ECORODOVIAS,ON NM,0.074,2.106,333657615
...,...,...,...,...,...,...
ENGI11,Utilidade Públ / Energ Elétrica,ENERGISA,UNT N2,0.545,9.638,248477689
ENEV3,Utilidade Públ / Energ Elétrica,ENEVA,ON NM,0.994,9.638,1561135009
EGIE3,Utilidade Públ / Energ Elétrica,ENGIE BRASIL,ON NM,0.488,9.638,255208461


In [ ]:
df_ibov.dtypes

segment         object
cod             object
asset           object
type            object
part            object
partAcum        object
theoricalQty    object
dtype: object

In [ ]:
from sqlalchemy.sql.sqltypes import VARCHAR
#inserir no tabela
dtype = {'cod': sql.VARCHAR(10),'segment':sql.VARCHAR(100),'asset':sql.VARCHAR(20),'type':sql.VARCHAR(10),'part':sql.FLOAT,'partAcum':sql.FLOAT,'theoricalQty': sql.INT} 
df_ibov.to_sql('ibov_b3',sql_conn,schema='public',if_exists='append',index = False, method="multi",chunksize=999,dtype= dtype)

#importar dados do IFIX

In [ ]:
#buscar index do ifix
index_ifix = getEmpIndex(seg=2,index='IFIX')

In [ ]:
#tranformar em dataframe
df_ifix = pd.DataFrame.from_dict(index_ifix['results'])

In [ ]:
#converter colunas para formato desejado
df_ifix['part']= df_ifix['part'].str.replace(',', '.',regex=True).astype(float)
df_ifix['partAcum'] = df_ifix['partAcum'].str.replace(',', '.',regex=True).astype(float)
df_ifix['theoricalQty'] = df_ifix['theoricalQty'].str.replace('.', '',regex=True).astype(int)

In [ ]:
df_ifix.dtypes

segment          object
cod              object
asset            object
type             object
part            float64
partAcum        float64
theoricalQty      int64
dtype: object

In [ ]:
from sqlalchemy.sql.sqltypes import VARCHAR
#Criar tabela e inserir os dados
dtype = {'cod': VARCHAR(10),
         'segment':VARCHAR(100),
         'asset':VARCHAR(20),
         'type':VARCHAR(10),
         'part':FLOAT,
         'partAcum':FLOAT,
         'theoricalQty': INT} 
df_ifix.to_sql('ifix_b3',sql_conn,schema='public',if_exists='append',index = False, method="multi",chunksize=999,dtype= dtype)

In [ ]:
ifix_cursor = sql_conn.execute("SELECT * FROM public.ifix_b3")  
for r in ifix_cursor:  
    print(r)

('Financ e Outros / Fundos', 'CACR11', 'CARTESIA FII', 'CI  ER', 0.162, 100.0, 1807434.0)
('Financ e Outros / Fundos', 'BPFF11', 'FII ABSOLUTO', 'CI  ER', 0.273, 100.0, 4492326.0)
('Financ e Outros / Fundos', 'AFHI11', 'FII AFHI CRI', 'CI', 0.259, 100.0, 3019453.0)
('Financ e Outros / Fundos', 'ALZR11', 'FII ALIANZA', 'CI', 0.869, 100.0, 8310805.0)
('Financ e Outros / Fundos', 'ARCT11', 'FII ARCTIUM', 'CI', 0.324, 100.0, 3708168.0)
('Financ e Outros / Fundos', 'ARRI11', 'FII ATRIO', 'CI  ER', 0.095, 100.0, 11782610.0)
('Financ e Outros / Fundos', 'AIEC11', 'FII AUTONOMY', 'CI  ER', 0.32, 100.0, 4824987.0)
('Financ e Outros / Fundos', 'BARI11', 'FII BARIGUI', 'CI  ER', 0.395, 100.0, 4634191.0)
('Financ e Outros / Fundos', 'BBPO11', 'FII BB PRGII', 'CI  ER', 1.294, 100.0, 15919690.0)
('Financ e Outros / Fundos', 'BCFF11', 'FII BC FFII', 'CI  ER', 1.549, 100.0, 25186378.0)
('Financ e Outros / Fundos', 'BRCR11', 'FII BC FUND', 'CI  ER', 1.527, 100.0, 26638202.0)
('Financ e Outros / Fundos'

In [ ]:
df_ifix  = ifix_cursor = sql_conn.execute("SELECT * FROM public.ifix_b3")  

In [ ]:
#criar index na tabela 
sql_conn.execute(" ALTER TABLE PUBLIC.ifix_b3 add CONSTRAINT pk_ifix_b3 PRIMARY KEY (cod)")

In [ ]:
#select tabela converter para dataframe
df_ifix = pd.read_sql(""" SELECT * FROM public.ifix_b3 """, sql_conn)

In [ ]:
#select com where e converter para dataframe
df_ifix_one = pd.read_sql(""" SELECT * FROM public.ifix_b3  where "cod" = 'ARCT11' """, sql_conn)

In [ ]:
df_ifix_one

,segment,cod,asset,type,part,partAcum,theoricalQty
0,Financ e Outros / Fundos,ARCT11,FII ARCTIUM,CI,0.324,100.0,3708168


In [ ]:
df_ifix

,segment,cod,asset,type,part,partAcum,theoricalQty
0,Financ e Outros / Fundos,CACR11,CARTESIA FII,CI ER,0.162,100.0,1807434
1,Financ e Outros / Fundos,BPFF11,FII ABSOLUTO,CI ER,0.273,100.0,4492326
2,Financ e Outros / Fundos,AFHI11,FII AFHI CRI,CI,0.259,100.0,3019453
3,Financ e Outros / Fundos,ALZR11,FII ALIANZA,CI,0.869,100.0,8310805
4,Financ e Outros / Fundos,ARCT11,FII ARCTIUM,CI,0.324,100.0,3708168
...,...,...,...,...,...,...,...
103,Financ e Outros / Fundos,XPIN11,FII XP INDL,CI,0.517,100.0,7150422
104,Financ e Outros / Fundos,XPLG11,FII XP LOG,CI ER,2.533,100.0,27101061
105,Financ e Outros / Fundos,XPML11,FII XP MALLS,CI,1.934,100.0,21133278
106,Financ e Outros / Fundos,XPPR11,FII XP PROP,CI ER,0.246,100.0,7316171


#Buscar todas as empresas listadas na B3

In [ ]:
def getAllCompanyB3(pageN=1,pageS=9999):
  
    params = {"language":"pt-br","pageNumber":pageN,"pageSize":pageS}
    # codifica os parâmetros em base64
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params)
    string = string.decode()
    r = requests.get('https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetInitialCompanies/'+ string)

    return r.json()

In [ ]:
#resultado da API
result = getAllCompanyB3()

In [ ]:
df_allCompany = pd.DataFrame.from_dict(result['results'])

In [ ]:

#converter colunas para formato desejado
df_allCompany['codeCVM']= df_allCompany['codeCVM'].str.replace(',', '.',regex=True).astype(int)
df_allCompany['marketIndicator']= df_allCompany['marketIndicator'].str.replace(',', '.',regex=True).astype(int)
df_allCompany['dateListing'] = df_allCompany['dateListing'].str.replace('9999', '1900',regex=True).astype('datetime64[ns]')
df_allCompany['type'] = df_allCompany['type'].str.replace('.', '',regex=True).astype(int)
#df_allCompany['dateListing'] = df_allCompany['dateListing'].apply(lambda x: (parser.parse(x.replace('9999','1900')))).astype('datetime64[ns]')

In [ ]:
df_allCompany.dtypes

codeCVM                     int64
issuingCompany             object
companyName                object
tradingName                object
cnpj                       object
marketIndicator             int64
typeBDR                    object
dateListing        datetime64[ns]
status                     object
segment                    object
segmentEng                 object
type                        int64
market                     object
dtype: object

In [ ]:
df_allCompany.rename(columns = {'codeCVM':'codecvm'}, inplace = True)

In [ ]:
from sqlalchemy.sql.sqltypes import VARCHAR,INT
#inserir no tabela
dtype = {'codecvm': INT,
         'issuingCompany':VARCHAR(10),
         'companyName':VARCHAR(200),
         'tradingName':VARCHAR(50),
         'marketIndicator':INT,
         'typeBDR':VARCHAR(20),
         'dateListing':DateTime,
         'status': VARCHAR(1),
         'segment':	VARCHAR(100),
         'segmentEng':VARCHAR(100),
         'type':INT,
         'market':VARCHAR(20)
         } 

df_allCompany.to_sql('company_b3',sql_conn,schema='public',if_exists='append',index = False, method="multi",chunksize=999,dtype= dtype)

In [ ]:
pd.read_sql(" select codecvm from PUBLIC.company_b3 ",sql_conn)

,codecvm,issuingCompany,companyName,tradingName,cnpj,marketIndicator,typeBDR,dateListing,status,segment,segmentEng,type,market
0,25291,RRRP,3R PETROLEUM ÓLEO E GÁS S.A,3R PETROLEUM,12091809000155,18,,2020-09-11,A,Exploração. Refino e Distribuição,Exploration. Refining and Distribution,1,NM
1,908830,TZRD,3Z REALTY DESENVOLVIMENTO IMOBILIARIO S/A,3ZREALTY,8145855000112,1,,1900-12-31,A,Não Classificados,Not Classified,7,
2,16284,QVQP,524 PARTICIPACOES S.A.,524 PARTICIP,1851771000155,7,,1998-11-09,A,Outros,Others,1,MB
3,904110,GRDR,A GERADORA ALUGUEL DE MAQUINAS S.A.,GERADORA,33845322000190,1,,1900-12-31,A,Não Classificados,Not Classified,7,
4,901082,BODY,A BODYTECH PARTICIPACOES S.A.,BODYTECH,7737623000190,1,,1900-12-31,A,Não Classificados,Not Classified,7,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2251,914804,ZZDT,Z.A DIGITAL SP SIST. ESTACIONAMENTO ROTARIVO S.A.,Z.A DIGITAL,35993098000100,1,,1900-12-31,A,Não Classificados,Not Classified,7,
2252,24317,ZAMP,ZAMP S.A.,ZAMP S.A.,13574594000196,18,,2017-12-14,A,Restaurante e Similares,Restaurants and Similars,1,NM
2253,915249,ZTIX,ZATIX TECNOLOGIA S.A.,ZATIX TECNOL,3585974000172,1,,1900-12-31,A,Não Classificados,Not Classified,7,
2254,909655,ZMPT,ZEMA CIA. DE PETROLEO,ZEMA,647154000170,1,,1900-12-31,A,Não Classificados,Not Classified,7,


In [ ]:
#criar index
#criar index na tabela 
sql_conn.execute(" ALTER TABLE PUBLIC.company_b3 add CONSTRAINT pk_company_b3 PRIMARY KEY (codecvm) ")

#buscar detalhes das empresas 

In [ ]:
def getDetailCompany(codeCVM):
  
    params = {"codeCVM":codeCVM,"language":"pt-br"}
    # codifica os parâmetros em base64
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params)
    string = string.decode()
    r = requests.get('https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetDetail/'+ string)

    return r.json()

In [ ]:
sql = "SELECT codecvm from PUBLIC.company_b3  WHERE codecvm != 26700 AND codecvm NOT IN ( SELECT codecvm FROM PUBLIC.detail_company_b3) LIMIT 500 "

cvms = pd.read_sql(sql,sql_conn)

In [ ]:
cvms.shape

(457, 1)

In [ ]:
from sqlalchemy.sql.sqltypes import VARCHAR,INT,FLOAT,DateTime,TEXT,NUMERIC,JSON
dtype = {'issuingcompany':VARCHAR(10),
         'companyName':VARCHAR(200),
         'tradingName':VARCHAR(100),
         'CNPJ':VARCHAR(20),
         'industryClassification': VARCHAR(200),
         'industryClassificationEng': VARCHAR(200),
         'activity':VARCHAR(200),
         'website':VARCHAR(100),
         'hasQuotation':VARCHAR(100),
         'status': VARCHAR(1),
         'marketIndicator':INT,
         'institutionCommon':VARCHAR(100),
         'institutionPreferred':VARCHAR(100),
        'code':	VARCHAR(10),
        'codecvm': INT,
        'lastdate':DateTime,
        'otherCodes':	JSON,
        'hasEmissions':Boolean,
        'hasBDR':Boolean,
        'typeBDR':VARCHAR(100),
        'describleCategoryBVMF':VARCHAR(200)
      } 

In [ ]:
import time
from datetime import datetime

all_detail_company = pd.DataFrame()
err_cvm = []
print(datetime.today())
for i in cvms.index:
    cod = cvms['codecvm'][i]
    print(cod)
    time.sleep(1)
    result = getDetailCompany(cod)
    #separar em as chaves e depois os valores para montar o dataframe

    if (len(result) == 0):
       err_cvm.append(cod)
       continue

    values = result.values()
    keys = result.keys()

    df_detCompany = pd.DataFrame([values],columns=keys)

    #converter colunas para formato desejado
    df_detCompany['codeCVM']= df_detCompany['codeCVM'].str.replace(',', '.',regex=True).astype(int)
    df_detCompany['marketIndicator']= df_detCompany['marketIndicator'].str.replace(',', '.',regex=True).astype(int)
    df_detCompany['lastDate'] = df_detCompany['lastDate'].astype('datetime64[ns]')
    df_detCompany.rename(columns = {'codeCVM':'codecvm','issuingCompany':'issuingcompany'}, inplace = True)

    #df_allCompany['dateListing'] = df_allCompany['dateListing'].apply(lambda x: (parser.parse(x.replace('9999','1900')))).astype('datetime64[ns]')

    all_detail_company =  pd.concat([all_detail_company,df_detCompany])
    print(cod)
all_detail_company.to_sql('detail_company_b3',sql_conn,schema='public',if_exists='append',index = False, method="multi",chunksize=999,dtype=dtype)
    
print(datetime.today())
print('empresa não encontrada')
print(err_cvm)

2022-11-12 18:01:23.500812
913376
913376
913384
913384
913392
913392
913400
913400
913418
913418
913426
913426
913434
913434
913442
913442
913459
913459
913467
913467
913475
913475
913483
913483
913491
913491
913509
913509
913517
913517
913525
913525
913533
913533
913541
913541
913558
913558
913566
913566
913574
913574
913582
913582
913590
913590
913608
913608
913616
913616
913624
913624
913632
913632
913640
913640
913657
913657
913665
913665
913673
913673
913681
913681
913699
913699
913749
913749
913756
913756
913764
913764
913772
913772
913780
913780
913798
913798
913806
913806
913814
913814
913822
913822
913830
913830
913848
913848
913889
913889
913897
913897
913905
913905
913913
913913
913921
913921
913939
913939
913947
913947
913954
913954
913962
913962
913970
913970
913988
913988
913996
913996
914028
914028
914036
914036
914044
914044
914051
914051
914077
914077
914085
914085
914093
914093
914101
914101
914119
914119
914127
914127
914135
914135
914143
914143
914176
914176
914184


In [ ]:
#df_detCompany['otherCodes']=  df_detCompany['otherCodes'].apply(lambda x: ((x.replace("[","").replace("]",""))))
#import json
#df_detCompany['otherCodes'] = df_detCompany['otherCodes'].apply(json.dumps)


#importar dados dos fundos imobiliarios

In [ ]:
import requests
from base64 import b64encode
# Função para obter os proventos
def getFii(pageN=1,pageS=999):
  
    # cria os parâmetros com o trading name

    params = {"typeFund":7,"pageNumber":pageN,"pageSize":pageS}
    
    # codifica os parâmetros em base64
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params)
    string = string.decode()

    r = requests.get('https://sistemaswebb3-listados.b3.com.br/fundsProxy/fundsCall/GetListedFundsSIG/'+string)
    return r.json()

In [ ]:
fii = getFii()

In [ ]:
df_all_fii = pd.DataFrame.from_dict(fii['results'])

In [ ]:
df_all_fii.rename(columns = {'fundName':'fundname','companyName':'companyname'}, inplace = True)

In [ ]:
#from sqlalchemy.sql import VARCHAR
dtype = {'segment':VARCHAR(10),
         'acronym':VARCHAR(50),
          'fundname':VARCHAR(50),
         'companyname':VARCHAR(200),
         'cnpj':VARCHAR(20),
      } 

In [ ]:
#gravar no banco SQL
df_all_fii.to_sql('fii_b3',sql_conn,schema='public',if_exists='append',index = False, method="multi",chunksize=999,dtype=dtype)

Buscar detalhes dos fundos imobiliarios

In [ ]:
import requests
from base64 import b64encode
# Função para obter os proventos
def getDetailFii(acronym):
  
    # codFII: "ASRF"
    params =   {"typeFund":7,"identifierFund":acronym}
    # codifica os parâmetros em base64
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params)
    string = string.decode()

    r = requests.get('https://sistemaswebb3-listados.b3.com.br/fundsProxy/fundsCall/GetDetailFundSIG/'+string)
    return r.json()

In [ ]:
df = getDetailFii('KNHY')

In [ ]:
#buscar nome dos fii na base de dados
# buscar todos empresas cadastrada na base de dados para fazer o SPLIT no campo industryclassification !!
sql = " SELECT distinct(acronym) FROM PUBLIC.fii_b3 WHERE acronym NOT IN (SELECT distinct(acronym) FROM  PUBLIC.detail_fii_b3)  "
def_fii = pd.read_sql(sql,sql_conn)

In [ ]:
def_fii

,acronym


In [ ]:
from sqlalchemy.sql.sqltypes import VARCHAR,INT,FLOAT,DateTime,TEXT,NUMERIC,JSON
dtype = {'acronym': VARCHAR(10),
          'tradingname': VARCHAR(30),
          'tradingcode': VARCHAR(10),
          'tradingcodeothers': VARCHAR(50),
          'cnpj': VARCHAR(20),
          'classification': VARCHAR(200),
          'website': VARCHAR(100),
          'fundaddress': VARCHAR(200),
          'fundphonenumberddd': VARCHAR(20),
          'fundphonenumber': VARCHAR(20),
          'fundphonenumberfax': VARCHAR(20),
          'positionmanager':  VARCHAR(50),
          'managername':  VARCHAR(50),
          'companyaddress': VARCHAR(200),
          'companyphonenumberddd': VARCHAR(10),
          'companyphonenumber': VARCHAR(20),
          'companyphonenumberfax': VARCHAR(20),
          'companyemail': VARCHAR(100),
          'companyname': VARCHAR(100),
          'quotacount': INT,
          'quotadateapproved': DateTime,
          'typefnet': VARCHAR(50),
          'codes': JSON,
          'codesother': VARCHAR(50),
          'segment': VARCHAR(50)
          }

In [ ]:

#key = list(df_fii.columns)
#value = [x.lower() for x in list(df_fii.columns)]
##columns = pd.DataFrame([value],columns=key).to_dict('records')[0]

import time
from datetime import datetime

err_det_fii = []
print(datetime.today())
for i in def_fii.index:
    cod = def_fii['acronym'][i]
    print(cod)
    time.sleep(2)
    #buscar detalhes dos fundos
    result = getDetailFii(cod)
  

    if (len(result['detailFund']) == 0):
       err_det_fii.append(cod)
       continue
    #renomear colunas com em lower case
    columns = { 'acronym': 'acronym',
               'tradingName': 'tradingname',
              'tradingCode': 'tradingcode',
              'tradingCodeOthers': 'tradingcodeothers',
              'cnpj': 'cnpj',
              'classification': 'classification',
              'webSite': 'website',
              'fundAddress': 'fundaddress',
              'fundPhoneNumberDDD': 'fundphonenumberddd',
              'fundPhoneNumber': 'fundphonenumber',
              'fundPhoneNumberFax': 'fundphonenumberfax',
              'positionManager': 'positionmanager',
              'managerName': 'managername',
              'companyAddress': 'companyaddress',
              'companyPhoneNumberDDD': 'companyphonenumberddd',
              'companyPhoneNumber': 'companyphonenumber',
              'companyPhoneNumberFax': 'companyphonenumberfax',
              'companyEmail': 'companyemail',
              'companyName': 'companyname',
              'quotaCount': 'quotacount',
              'quotaDateApproved': 'quotadateapproved',
              'typeFNET': 'typefnet',
              'codes': 'codes',
              'codesOther': 'codesother',
              'segment': 'segment' }

    df_fii = pd.DataFrame([result['detailFund']])
    df_fii.rename(columns = columns, inplace = True)
    df_fii['cnpj'] =  df_fii['cnpj'].astype(int)
    df_fii['quotacount'] =  df_fii['quotacount'].astype(int)
    df_fii['quotadateapproved'] = pd.to_datetime( df_fii['quotadateapproved'], errors ='coerce')
    df_fii.fillna('')
    df_fii.to_sql('detail_fii_b3',sql_conn,schema='public',if_exists='append',index = False, method="multi",dtype=dtype)
    
      
print(datetime.today())
print('não existem detalhes')
print(err_det_fii)


2022-11-18 21:41:33.800859
ABCP
FLMA
MOFF
LFTT
HUSI
CJFI
SHDP
BCRI
HLOG
RBRY
JFLL
SFND
CCME
GURB
VIDS
FIIP
BVAR
XPCI
PLOG
HGCR
VLJS
TEPP
RBVA
FMOF
GGRC
KNHY
BTSG
SIGR
RBHG
VLIQ
VCJR
VDSV
TSNC
RBLG
CTNP
SOLR
RBRM
AFHI
MTOF
OURE
MMPD
AURB
BRCR
DEVA
CYCR
PRSN
JPPA
ARRI
RCKF
FIIB
BRIP
ALZM
CCRF
KISU
YUFI
NAVT
PRTS
FPAB
MORC
JCDB
TCIN
IRDM
FAMB
XPIN
HTMX
FAGL
EVBI
ZAVI
HCHG
CXCE
TRXF
PRSV
JTPR
QAMI
HCRI
XPPR
IDFI
FCFL
CEOC
TRXB
KFOF
VSLH
RRCI
SPVJ
CPFF
FLRP
IBCR
RVBI
FEXC
BARI
BMLC
BRSE
DVFF
IRIM
EDGA
HGFF
MVFI
NEWL
RBIR
HCTR
RECX
LRDI
PQAG
BRCO
BBRC
TGAR
FIGS
BZEL
RPRI
PNDL
BBPO
2022-11-18 21:49:07.211682
não existem detalhes
[]


# Atualizar o Setor/Subsetor e Segmento da empresas listada na B3

In [ ]:
# buscar todos empresas cadastrada na base de dados para fazer o SPLIT no campo industryclassification !!

sql = " SELECT * FROM PUBLIC.detail_company_b3  WHERE industryclassification IS NOT NULL "

empresas = pd.read_sql(sql,sql_conn)

In [ ]:
empresas[["setor","subsetor","segmento"]] = empresas["industryclassification"].str.split("/",expand=True)

In [ ]:
empresas

,issuingcompany,companyname,tradingname,cnpj,industryclassification,industryclassificationEng,activity,website,hasquotation,status,...,othercodes,hasemissions,hasbdr,typebdr,describleCategorybvmf,datelisting,lastdate,setor,subsetor,segmento
0,WEGE,WEG S.A.,WEG,84429695000111,Bens Industriais / Máquinas e Equipamentos / M...,None,A WEG SA é uma sociedade de participação não o...,www.weg.net/br,None,A,...,"[{'code': 'WEGE3', 'isin': 'BRWEGEACNOR0'}]",False,False,None,None,2022-11-11 23:41:58,2022-11-11 23:41:58,Bens Industriais,Máquinas e Equipamentos,Motores . Compressores e Outros
1,ENER,ENERGISA MATO GROSSO DO SUL - DIST DE ENERGIA ...,ENERSUL,15413826000150,Utilidade Pública / Energia Elétrica / Energia...,None,Distribuição de energia elétrica,www.enersul.com.br,None,A,...,None,False,False,None,None,2022-11-11 23:18:00,2022-11-11 23:18:00,Utilidade Pública,Energia Elétrica,Energia Elétrica
2,ETER,ETERNIT S.A.,ETERNIT,61092037000181,Bens Industriais / Construção e Engenharia / P...,None,Industrialização e comercialização de produtos...,www.eternit.com.br/ri,None,A,...,"[{'code': 'ETER3', 'isin': 'BRETERACNOR3'}]",False,False,None,None,2022-11-11 23:18:53,2022-11-11 23:18:53,Bens Industriais,Construção e Engenharia,Produtos para Construção
3,EUCA,EUCATEX S.A. INDUSTRIA E COMERCIO,EUCATEX,56643018000166,Materiais Básicos / Madeira e Papel / Madeira,None,Indústria e Comércio de chapas de fibra de mad...,www.eucatex.com.br/ri,None,A,...,"[{'code': 'EUCA3', 'isin': 'BREUCAACNOR1'}, {'...",False,False,None,None,2022-11-11 23:18:55,2022-11-11 23:18:55,Materiais Básicos,Madeira e Papel,Madeira
4,FIGE,INVESTIMENTOS BEMGE S.A.,INVEST BEMGE,01548981000179,Outros / Outros / Outros,None,Investimentos,www.investimentosbemge.com.br,None,A,...,"[{'code': 'FIGE3', 'isin': 'BRFIGEACNOR6'}, {'...",False,False,None,None,2022-11-11 23:23:25,2022-11-11 23:23:25,Outros,Outros,Outros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,TCUL,TERRA DE CULTIVO INDS. DE FERT. E RACOES LTDA,TERRAINDSLTD,06105854000119,Não Classificados / Não Classificado / Não Cla...,None,None,,None,A,...,None,False,False,None,None,NaT,2022-11-11 23:38:14,Não Classificados,Não Classificado,Não Classificados
2030,DETR,TRAVESSIA E DELTA SEC. DE CREDITOS MERCANTIS S.A.,TRAVDELTASEC,41719223000190,Não Classificados / Não Classificado / Não Cla...,None,None,,None,A,...,None,False,False,None,None,NaT,2022-11-11 23:38:58,Não Classificados,Não Classificado,Não Classificados
2031,ALEX,ALEX ENERGIA PARTICIPACOES S.A.,ALEXENERGIAP,31908068000105,Não Classificados / Não Classificado / Não Cla...,None,None,,None,A,...,None,False,False,None,None,NaT,2022-11-11 23:04:38,Não Classificados,Não Classificado,Não Classificados
2032,THAP,THALASSIUS A033.21 PARTICIPAÇÕES S.A,THALASSIUSA0,42152864000179,Não Classificados / Não Classificado / Não Cla...,None,None,,None,A,...,None,False,False,None,None,NaT,2022-11-11 23:38:22,Não Classificados,Não Classificado,Não Classificados


In [ ]:
from sqlalchemy.sql.sqltypes import VARCHAR,INT,FLOAT,DateTime,TEXT,NUMERIC,JSON,Boolean
dtype = {'issuingcompany': VARCHAR(10),
         'companyname':VARCHAR(200),
         'tradingname':VARCHAR(100),
         'cnpj': VARCHAR(20),
         'industryclassification': VARCHAR(200),
         'industryclassificationEng': VARCHAR(200),
         'activity':VARCHAR(200),
         'website':VARCHAR(100),
         'hasquotation':VARCHAR(100),
         'status': VARCHAR(1),
         'marketindicator':INT,
         'institutioncommon':VARCHAR(100),
         'institutionpreferred':VARCHAR(100),
         'code':	VARCHAR(10),
         'codecvm': INT,
         'lastdate':DateTime,
         'othercodes':	JSON,
         'hasemissions':Boolean,
         'hasbdr':Boolean,
         'typebdr':VARCHAR(100),
         'describlecategorybvmf': VARCHAR(200),
         'setor':VARCHAR(100),
         'subsetor':VARCHAR(100),
         'segmento':VARCHAR(100)
      } 
#empresas.to_sql('detail_company_b3',sql_conn,schema='public',if_exists='replace',index = False, method="multi",dtype=dtype)

In [ ]:
# formar tabela de setor 
sql = " SELECT distinct(setor) as descricao FROM PUBLIC.detail_company_b3  WHERE industryclassification IS NOT NULL order by setor "
df_setores = pd.read_sql(sql,sql_conn)

In [ ]:
from sqlalchemy.sql.sqltypes import VARCHAR,INT
#criar uma coluna a partir do index
df_setores = df_setores['descricao'].str.strip()
df_setores = df_setores.rename_axis('codigo').reset_index()
#deixar o index a partir de 1
df_setores['codigo'] = df_setores['codigo']+1

dtype = {'codigo':INT,
         'descricao':VARCHAR(100)
}

#criar a tabela no banco
df_setores.to_sql('setores_b3',sql_conn,schema='public',if_exists='replace',index = False, method="multi",dtype=dtype)

In [ ]:
# formar tabela de subserto
sql = " SELECT distinct(subsetor) as descricao FROM PUBLIC.detail_company_b3  WHERE industryclassification IS NOT NULL order by subsetor "
df_subsetores = pd.read_sql(sql,sql_conn)

In [ ]:
df_subsetores = df_subsetores['descricao'].str.strip().replace('Não Classificados','Não Classificado').drop_duplicates()
df_subsetores = df_subsetores.rename_axis('codigo').reset_index()
df_subsetores['codigo'] = df_setores['codigo']+1
dtype = {'codigo':INT,
         'descricao':VARCHAR(100)
}
#criar a tabela no banco
df_setores.to_sql('subsetores_b3',sql_conn,schema='public',if_exists='replace',index = False, method="multi",dtype=dtype)

In [ ]:
# criar tabela de segmentos
sql = " SELECT distinct(segmento) as descricao FROM PUBLIC.detail_company_b3  WHERE industryclassification IS NOT NULL order by segmento "
df_segmento = pd.read_sql(sql,sql_conn)

In [ ]:
#df_segmento.groupby(['descricao']).size().sort_values()
#remover duplicadas e limpar espaço em brancos
df_segmento['descricao'] = df_segmento['descricao'].str.strip().drop_duplicates()
###
df_segmento = df_segmento.rename_axis('codigo').reset_index()
df_segmento['codigo'] = df_segmento['codigo'] + 1
dtype = {'codigo':INT,
          'descricao':VARCHAR(100)
}
# #criar a tabela no banco
df_segmento.to_sql('segmentos_b3',sql_conn,schema='public',if_exists='replace',index = False, method="multi",dtype=dtype)


#buscar dados dos dividendos dos ativos

In [ ]:
def tipo_acao(tipo):
      if tipo == 'ON':
        vTipo = 3
      elif tipo == 'PN':
        vTipo = 4
      elif tipo == 'PNA':
        vTipo = 5
      elif tipo == 'PNB':
        vTipo = 6
      elif tipo == 'UNIT':
        vTipo = 11
      else:
         vTipo = ''

      return vTipo

In [ ]:
# Função para obter os proventos
def dividendos(tradingname):
    # cria os parâmetros com o trading name
    params = {"language":"pt-br","pageNumber":1,"pageSize":99999,"tradingName":tradingname}
    # codifica os parâmetros em base64
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params)
    string = string.decode()
    r = requests.get('https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/'+ string)

    return r.json()

In [ ]:
# buscar tabelas de company
sql = " SELECT codecvm, tradingname, issuingcompany FROM public.company_b3 WHERE codecvm not in (SELECT  distinct(codecvm) FROM PUBLIC.dividendos_b3) AND codecvm IN  (SELECT  distinct(codecvm) FROM PUBLIC.detail_company_b3 WHERE code IS NOT null) order by codecvm "
df_company = pd.read_sql(sql,sql_conn)
df_company['tradingname']   = df_company['tradingname'].str.replace('/','').str.replace(' ','')

In [ ]:
#list_erro =  ['HERCULES', 'HOTEISOTHON', 'ESTRELA', 'IGUATEMIS.A', 'NORDONMET', 'SPTURIS', 'WETZELSA', 'SANSUY', 'HAGASA', 'CEMEPE', 'SUZANOS.A.', 'DIBENSLSG', 'CIMS', 'P.ACUCAR-CBD', 'LITEL', 'ALLNORTE', 'FERCATLANT', '524PARTICIP', 'SUL116PART', 'SUDESTESA', 'EQTLMARANHAO', 'UPTICK', 'YBYRASA', 'BNDESPAR', 'RUMOS.A.', 'BETAPART', 'MMXMINER', 'ALLPAULISTA', 'MRSLOGIST', 'GAMAPART', 'CONFIANZASA', 'CIBRASEC', 'PROMPTPART', 'RBCAPITALRES', 'PADTEC', 'WTORREPIC', 'MENEZESCORT', 'DTCOM-DIRECT', 'BRAZILIANSC', 'INVEPAR', 'CABINDAPART', 'CONCRIOTER', 'PROMAN', 'BIOMM', 'ITAPEBI', 'SANESALTO', 'BRADESCOLSG', 'BRAZILREALT', 'TERMOPE', 'BRPR56SEC']

In [ ]:
df_company = df_company.query('tradingname not in @list_erro')

In [ ]:

import time
from datetime import datetime

all_div = pd.DataFrame()
err_div = []
print(datetime.today())
for i in df_company.index:
    cod = df_company['tradingname'][i]
    print(cod)
    time.sleep(2)
    result = dividendos(cod)
    
    #separar em as chaves e depois os valores para montar o dataframe

    if (len(result['results']) == 0):
       err_div.append(cod)
       continue

    df_proventos = pd.DataFrame(result['results'])
        #transformar as colunas nos types necessario

    #renomear as colunas
    df_proventos.rename(columns = {'typeStock':'typestock',
                                  'dateApproval':'dateapproval',
                                   'valueCash':'valuecash',
                                   'corporateAction':'corporateaction',
                                   'lastDatePriorEx':'lastdatepriorex',
                                   'dateClosingPricePriorExDate':'dateclosingpricepriorexdate',
                                   'closingPricePriorExDate':'closingpricepriorexdate',
                                   'quotedPerShares':'quotedpershares',
                                    'corporateActionPrice':'corporateactionprice',
                                   'lastDateTimePriorEx':'lastdatetimepriorex'}, inplace = True)
    
    df_proventos['codecvm'] = df_company['codecvm'][i]
    df_proventos['tradingname'] = df_company['tradingname'][i]
    df_proventos['code'] = df_proventos['typestock'].apply(lambda x: (df_company['issuingcompany'][i]+str(tipo_acao(x))))
    df_proventos['valuecash'] = df_proventos['valuecash'].str.replace('.','',regex=True).str.replace(',','.',regex=True).astype(float)
    
    df_proventos['closingpricepriorexdate'] = df_proventos['closingpricepriorexdate'].str.replace('.','',regex=True).str.replace(',','.',regex=True).astype(float)
    df_proventos['corporateactionprice'] = df_proventos['corporateactionprice'].str.replace('.','',regex=True).str.replace(',','.',regex=True).astype(float)
    
    #df_proventos['lastdatepriorex'] = df_proventos['lastdatepriorex'].astype('datetime64[ns]')
    df_proventos['lastdatepriorex'] = pd.to_datetime( df_proventos['lastdatepriorex'] , errors ='coerce')
    try:
       #df_proventos['dateclosingpricepriorexdate'] = df_proventos['dateclosingpricepriorexdate'].astype('datetime64[ns]')
       df_proventos['dateclosingpricepriorexdate'] = pd.to_datetime( df_proventos['dateclosingpricepriorexdate'] , errors ='coerce')
    except Exception:
      df_proventos['dateclosingpricepriorexdate'] = ''
       	
    #df_proventos['lastdatetimepriorex'] =     df_proventos['lastdatetimepriorex'].astype('datetime64[ns]')

    df_proventos['lastdatetimepriorex'] = pd.to_datetime( df_proventos['lastdatetimepriorex'] , errors ='coerce')

    #df_proventos['dateapproval'] = df_proventos['dateapproval'].astype('datetime64[ns]')
    df_proventos['dateapproval'] = pd.to_datetime( df_proventos['dateapproval'] , errors ='coerce')

    df_proventos.fillna('')

    #all_div =  pd.concat([all_div,df_proventos])

    df_proventos.to_sql('dividendos_b3',sql_conn, schema='public',if_exists='append',index = False, method="multi", dtype=dtype)

    # #criar a tabela no banco
#all_div.to_sql('dividendos_b3',sql_conn, schema='public',if_exists='append',index = False, method="multi", dtype=dtype)   
print(datetime.today())
print('não existem dividendos')
print(err_div)


2022-11-18 16:38:15.771962
BKRSHORTTR
BKR13YRTR
BKR20YRTRS
ABDENSILV
ABDENGOLD
PMCO1T5YRT
PMCOBROAD
PMCO15YRTP
PMCOIGRCPI
PCOM0T5HY
PCOM25YRZC
BKRCOREHDV
BKRGBSTAMP
BKRCHINAA
BKRSAUDARAB
BKRSPSM600V
BKRCODISCRT
BKRCYBTECH
BKRMORTGAGE
BKREURPFNC
BKRRESMULTS
BKRSELFDRIV
BKRCOSTAPLE
BKRGENOIMM
BKRLOWCARBO
BKRGBMMPRD
BKRROBTAIM
BKRGBSDVGO
BKRUSINFRA
BKRUSUTILT
GXSP500CC
GXRUS2000CC
GXSUDIVREIT
GXMSCIGREEC
GXMSCICHIFL
GXCANNABIS
JPBTBCANAD
JPBTBREIT
JPRETUINTL
JPBTBRETUR
JPBTBUSEQ
JPQLTFACT
JPDIVRETUS
JPDVUSEME
JPUSVLFAC
JPDVUSMID
JPUSMOFAC
JPDVUSSM
GXBLOCKCHAI
GXCLEANCH
PIMCOMFEME
WTCLOUCOMP
VANECKEMERG
VANECKANGL
VANECKPREF
STONECO
XPINC
INTERCO
GOLDTRUST
MSCIEAFE
MSCIEMGMARK
MSCIASIAJP
SELECTDIVID
SILVERTRUST
RUSSELL2000
CORESPTOTA
MSCIUSMVOL
MSCIUK
MSCIGERMANY
MSCIMEXICO
MSCISPAIN
USAEROSPACE
LATINAMER40
CORESP500
MSCIACWI
COREMSCIEAFE
COREMSCIEMK
USTECHNOLOG
MSCIJAPAN
USFINANCIAL
MSCICHINA
GLOBALINFRA
EXPONTECHNL
TRUSTMSCIUS
TRTMSCIEAFE
INCESGAWAR
ICEBIOTECH
COREMIDCAP
CORESMALLCAP
PRE

KeyboardInterrupt: ignored

In [ ]:
#all_div.to_sql('dividendos_b3',sql_conn, schema='public',if_exists='append',index = False, method="multi", dtype=dtype) 